**This notebook is to train and test the CNN-driven advection scheme.**

This notebook perform loading the velocity field, implementing the machine-learned scheme, training the scheme, wallclock time measurement, and visualization.

In [1]:
## Import packages ##

#import Pkg
#Pkg.add("Plots")
using Plots
#Pkg.add("BenchmarkTools")
using BenchmarkTools
#Pkg.add("Flux")
using Flux
#Pkg.add("CSV")
using CSV
#Pkg.add("DelimitedFiles")
using DelimitedFiles
#Pkg.add("Statistics")
using Statistics
#Pkg.add("BSON")
using BSON: @save
using BSON: @load
#Pkg.add("Random")
using Random
#Pkg.add("CUDA")
using CUDA

In [2]:
# When we trained the model with a single dataset, we used this code.
vel1_GEOS_Array = readdlm( "Training_dataset/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_1x_1x.csv", ',', Float32);
scalar1_GEOS_Array = readdlm("Training_dataset/VL_GEOS_Jan_2019_NASA_GMAO_10_U_1x_1x.csv", ',', Float32);

xdim = size(vel1_GEOS_Array, 1);
nstep = size(vel1_GEOS_Array, 2);

In [ ]:
# Sometimes, we wanted to check the maximum CFL number in the velocity field we used.
maximum(vel1_GEOS_Array*300/27034.3)

In [ ]:
## For training with the multiple datasets, we used this code.
## For 2-D demonstration, we trained the solver for (16dx, 64dt) resolution again with 7 different datasets, 
## including the orignial trainign dataset and the generalization testing sets.

vel1_GEOS_Array = readdlm( "Training_dataset/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar1_GEOS_Array = readdlm("Training_dataset/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
vel2_GEOS_Array = readdlm("Generalization_tests/Season_April/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar2_GEOS_Array = readdlm("Generalization_tests/Season_April/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
vel3_GEOS_Array = readdlm("Generalization_tests/Season_Jul/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar3_GEOS_Array = readdlm("Generalization_tests/Season_Jul/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
vel4_GEOS_Array = readdlm("Generalization_tests/Season_Oct/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar4_GEOS_Array = readdlm("Generalization_tests/Season_Oct/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
vel5_GEOS_Array = readdlm("Generalization_tests/Lat_29N_corrected/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar5_GEOS_Array = readdlm("Generalization_tests/Lat_29N_corrected/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
vel6_GEOS_Array = readdlm("Generalization_tests/Lat_45N_corrected/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar6_GEOS_Array = readdlm("Generalization_tests/Lat_45N_corrected/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
vel7_GEOS_Array = readdlm("Generalization_tests/Longitudinal/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar7_GEOS_Array = readdlm("Generalization_tests/Longitudinal/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
xdim = size(vel1_GEOS_Array, 1);
ydim = size(vel7_GEOS_Array, 1);
nstep = size(vel1_GEOS_Array, 2);

In [3]:
# Add noise and normalize the input scale.

Random.seed!(1)
history = (scalar1_GEOS_Array*Float32(1e7), vel1_GEOS_Array/15);
#history = (scalar1_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel1_GEOS_Array/15);
#history2 = (scalar2_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel2_GEOS_Array/15);
#history3 = (scalar3_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel3_GEOS_Array/15);
#history4 = (scalar4_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel4_GEOS_Array/15);
#history5 = (scalar5_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel5_GEOS_Array/15);
#history6 = (scalar6_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel6_GEOS_Array/15);
#history7 = (scalar7_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, ydim, nstep), vel7_GEOS_Array/15);

Depending on the resolutions, we could use the solver code with different kernel sizes. pgm_ml() is the code to integrate throughout the whole timesteps, while one_step_integrate() is the code for a single step integration, which is used in model training.

In [4]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 3 stencil (default) ##

function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep) #|> gpu
    history_2x_learned[:,:,1] = x[:,1,1,1] #|> gpu
    s1_input = x[:,:,:,1] #|> gpu
    s1_scale = zeros(Float32, xdim_total, 3) #|> gpu
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 3)) #|> gpu
        
        s1_scale[1,1] = s1_input[1] #|> gpu
        s1_scale[2:xdim_total,1] = s1_input[1:xdim_total-1] #|> gpu
        
        s1_scale[:,2] = s1_input[1:xdim_total] #|> gpu
        
        s1_scale[1:xdim_total-1,3] = s1_input[2:xdim_total] #|> gpu
        s1_scale[xdim_total,3] = s1_input[xdim_total] #|> gpu
        
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2) #|> gpu
        
        history_2x_learned[:,1,n+1] = s2_2x #|> gpu
        s1_input = reshape(s2_2x, (xdim_total,1,1)) #|> gpu
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [ ]:
## Integrating a single timestep using ML based advection solver ##
## This code is called for model training later ##
## Integrate function with 3 stencil (default) ##

function one_step_integrate(x, u1, model, xdim_total, dx, dt)
    ## Initialize
    s1_input = reshape(x, (xdim_total, 1, 1))
    #s1_scale = s1_input
    
    # learned solver
    coeff_estimated = reshape(model(hcat(s1_input, u1)), (xdim_total, 2, 3))
    su = s1_input
    s1_bc = vcat( [su[1]], [su[i] for i in 1:xdim_total], [su[xdim_total]])
    s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2]), xdim_total, 3)
    s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
    
    s1_input = reshape(s2_2x, (xdim_total,1,1))
    return s2_2x
end

In [92]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 5 stencil ##

function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep) #|> gpu
    history_2x_learned[:,:,1] = x[:,1,1,1] #|> gpu
    s1_input = x[:,:,:,1] #|> gpu
    s1_scale = zeros(Float32, xdim_total, 5) #|> gpu
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 5)) #|> gpu
        
        s1_scale[1:2,1] .= s1_input[1] #|> gpu
        s1_scale[3:xdim_total,1] = s1_input[1:xdim_total-2] #|> gpu
        
        s1_scale[1,2] = s1_input[1] #|> gpu
        s1_scale[2:xdim_total,1] = s1_input[1:xdim_total-1] #|> gpu
        
        s1_scale[:,3] = s1_input[1:xdim_total] #|> gpu
        
        s1_scale[1:xdim_total-1,4] = s1_input[2:xdim_total] #|> gpu
        s1_scale[xdim_total,4] = s1_input[xdim_total] #|> gpu
        
        s1_scale[1:xdim_total-2,5] = s1_input[3:xdim_total] #|> gpu
        s1_scale[xdim_total-1:xdim_total,5] .= s1_input[xdim_total] #|> gpu
        
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2) #|> gpu
        
        history_2x_learned[:,1,n+1] = s2_2x #|> gpu
        s1_input = reshape(s2_2x, (xdim_total,1,1)) #|> gpu
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [ ]:
## Integrating a single timestep using ML based advection solver ##
## This code is called for model training later ##
## Integrate function with 5 stencil ##

function one_step_integrate(x, u1, model, xdim_total, dx, dt)
    ## Initialize
    s1_input = reshape(x, (xdim, 1, 1))
    #s1_scale = s1_input
    
    # learned solver
    coeff_estimated = reshape(model(hcat(s1_input, u1)), (xdim_total, 2, 5))
    su = s1_input
    s1_bc = vcat( [su[1]], [su[1]], [su[i] for i in 1:xdim_total], [su[xdim_total]], [su[xdim_total]])
    s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4]), xdim_total, 5)
    s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
    
    s1_input = reshape(s2_2x, (xdim_total,1,1))
    return s2_2x
end

In [94]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 9 stencil ##

function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep) #|> gpu
    history_2x_learned[:,:,1] = x[:,1,1,1] #|> gpu
    s1_input = x[:,:,:,1] #|> gpu
    s1_scale = zeros(Float32, xdim_total, 9) #|> gpu
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 9)) #|> gpu
        s1_scale[1:4,1] .= s1_input[1] #|> gpu
        s1_scale[5:xdim_total,1] = s1_input[1:xdim_total-4] #|> gpu
        
        s1_scale[1:3,2] .= s1_input[1] #|> gpu
        s1_scale[4:xdim_total,2] = s1_input[1:xdim_total-3] #|> gpu
        
        s1_scale[1:2,3] .= s1_input[1] #|> gpu
        s1_scale[3:xdim_total,3] = s1_input[1:xdim_total-2] #|> gpu
        
        s1_scale[1,4] = s1_input[1] #|> gpu
        s1_scale[2:xdim_total,4] = s1_input[1:xdim_total-1] #|> gpu
        
        
        s1_scale[:,5] = s1_input[1:xdim_total] #|> gpu
        
        
        s1_scale[1:xdim_total-1,6] = s1_input[2:xdim_total] #|> gpu
        s1_scale[xdim_total,6] = s1_input[xdim_total] #|> gpu
        
        s1_scale[1:xdim_total-2,7] = s1_input[3:xdim_total] #|> gpu
        s1_scale[xdim_total-1:xdim_total,7] .= s1_input[xdim_total] #|> gpu
        
        s1_scale[1:xdim_total-3,8] = s1_input[4:xdim_total] #|> gpu
        s1_scale[xdim_total-2:xdim_total,8] .= s1_input[xdim_total] #|> gpu
        
        s1_scale[1:xdim_total-4,9] = s1_input[5:xdim_total] #|> gpu
        s1_scale[xdim_total-3:xdim_total,9] .= s1_input[xdim_total] #|> gpu
        
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2) #|> gpu
        
        history_2x_learned[:,1,n+1] = s2_2x #|> gpu
        s1_input = reshape(s2_2x, (xdim_total,1,1)) #|> gpu
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [ ]:
## Integrating a single timestep using ML based advection solver ##
## This code is called for model training later ##
## Integrate function with 9 stencil ##

function one_step_integrate(x, u1, model, xdim_total, dx, dt)
    ## Initialize
    s1_input = reshape(x, (xdim, 1, 1))
    #s1_scale = s1_input
    
    # learned solver
    coeff_estimated = reshape(model(hcat(s1_input, u1)), (xdim_total, 2, 9))
    su = s1_input
    s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
    s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8]), xdim_total, 9)
    s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
    
    s1_input = reshape(s2_2x, (xdim_total,1,1))
    return s2_2x
end

In [16]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 17 stencil ##

function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep) |> gpu
    history_2x_learned[:,:,1] = x[:,1,1,1] |> gpu
    s1_input = x[:,:,:,1] |> gpu
    s1_scale = zeros(Float32, xdim_total, 17) |> gpu
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 17)) |> gpu
        
        s1_scale[1:8,1] .= s1_input[1] |> gpu
        s1_scale[9:xdim_total,1] = s1_input[1:xdim_total-8] |> gpu
        
        s1_scale[1:7,2] .= s1_input[1] |> gpu
        s1_scale[8:xdim_total,2] = s1_input[1:xdim_total-7] |> gpu
        
        s1_scale[1:6,3] .= s1_input[1] |> gpu
        s1_scale[7:xdim_total,3] = s1_input[1:xdim_total-6] |> gpu
        
        s1_scale[1:5,4] .= s1_input[1] |> gpu
        s1_scale[6:xdim_total,4] = s1_input[1:xdim_total-5] |> gpu
        
        s1_scale[1:4,5] .= s1_input[1] |> gpu
        s1_scale[5:xdim_total,5] = s1_input[1:xdim_total-4] |> gpu
        
        s1_scale[1:3,6] .= s1_input[1] |> gpu
        s1_scale[4:xdim_total,6] = s1_input[1:xdim_total-3] |> gpu
        
        s1_scale[1:2,7] .= s1_input[1] |> gpu
        s1_scale[3:xdim_total,7] = s1_input[1:xdim_total-2] |> gpu
        
        s1_scale[1,8] = s1_input[1] |> gpu
        s1_scale[2:xdim_total,8] = s1_input[1:xdim_total-1] |> gpu
        
        
        
        s1_scale[:,9] = s1_input[1:xdim_total] |> gpu
        
        
        
        s1_scale[1:xdim_total-1,10] = s1_input[2:xdim_total] |> gpu
        s1_scale[xdim_total,10] = s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-2,11] = s1_input[3:xdim_total] |> gpu
        s1_scale[xdim_total-1:xdim_total,11] .= s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-3,12] = s1_input[4:xdim_total] |> gpu
        s1_scale[xdim_total-2:xdim_total,12] .= s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-4,13] = s1_input[5:xdim_total] |> gpu
        s1_scale[xdim_total-3:xdim_total,13] .= s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-5,14] = s1_input[6:xdim_total] |> gpu
        s1_scale[xdim_total-4:xdim_total,14] .= s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-6,15] = s1_input[7:xdim_total] |> gpu
        s1_scale[xdim_total-5:xdim_total,15] .= s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-7,16] = s1_input[8:xdim_total] |> gpu
        s1_scale[xdim_total-6:xdim_total,16] .= s1_input[xdim_total] |> gpu
        
        s1_scale[1:xdim_total-8,17] = s1_input[9:xdim_total] |> gpu
        s1_scale[xdim_total-7:xdim_total,17] .= s1_input[xdim_total] |> gpu
        
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2) |> gpu
        
        history_2x_learned[:,1,n+1] = s2_2x |> gpu
        s1_input = reshape(s2_2x, (xdim_total,1,1)) |> gpu
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [ ]:
## Integrating a single timestep using ML based advection solver ##
## This code is called for model training later ##
## Integrate function with 17 stencil ##

function one_step_integrate(x, u1, model, xdim_total, dx, dt)
    ## Initialize
    s1_input = reshape(x, (xdim, 1, 1))
    #s1_scale = s1_input
    
    # learned solver
    coeff_estimated = reshape(model(hcat(s1_input, u1)), (xdim_total, 2, 17))
    su = s1_input
    s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16]), xdim_total, 17)
    s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
    
    s1_input = reshape(s2_2x, (xdim_total,1,1))
    return s2_2x
end

In [ ]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 31 stencil ##

function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 31))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], 
                [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16],
                [s1_bc[i] for i in 18:xdim_total+17],
                [s1_bc[i] for i in 19:xdim_total+18],
                [s1_bc[i] for i in 20:xdim_total+19],
                [s1_bc[i] for i in 21:xdim_total+20],
                [s1_bc[i] for i in 22:xdim_total+21],
                [s1_bc[i] for i in 23:xdim_total+22],
                [s1_bc[i] for i in 24:xdim_total+23],
                [s1_bc[i] for i in 25:xdim_total+24],
                [s1_bc[i] for i in 26:xdim_total+25],
                [s1_bc[i] for i in 27:xdim_total+26],
                [s1_bc[i] for i in 28:xdim_total+27],
                [s1_bc[i] for i in 29:xdim_total+28],
                [s1_bc[i] for i in 30:xdim_total+29],
                [s1_bc[i] for i in 31:xdim_total+30]), xdim_total, 31)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

In [ ]:
## Integrating a single timestep using ML based advection solver ##
## This code is called for model training later ##
## Integrate function with 31 stencil ##

function one_step_integrate(x, u1, model, xdim_total, dx, dt)
    ## Initialize
    s1_input = reshape(x, (xdim, 1, 1))
    #s1_scale = s1_input
    
    # learned solver
    coeff_estimated = reshape(model(hcat(s1_input, u1)), (xdim_total, 2, 31))
    su = s1_input
    s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]],
                [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]],
                [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
    s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16],
                [s1_bc[i] for i in 18:xdim_total+17],
                [s1_bc[i] for i in 19:xdim_total+18],
                [s1_bc[i] for i in 20:xdim_total+19],
                [s1_bc[i] for i in 21:xdim_total+20],
                [s1_bc[i] for i in 22:xdim_total+21],
                [s1_bc[i] for i in 23:xdim_total+22],
                [s1_bc[i] for i in 24:xdim_total+23],
                [s1_bc[i] for i in 25:xdim_total+24],
                [s1_bc[i] for i in 26:xdim_total+25],
                [s1_bc[i] for i in 27:xdim_total+26],
                [s1_bc[i] for i in 28:xdim_total+27],
                [s1_bc[i] for i in 29:xdim_total+28],
                [s1_bc[i] for i in 30:xdim_total+29],
                [s1_bc[i] for i in 31:xdim_total+30]), xdim_total, 31)
    s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
    
    s1_input = reshape(s2_2x, (xdim_total,1,1))
    return s2_2x
end

Now we generate arrays for storing training dataset.

In [5]:
## Generate arrays to store the input (scalar and velocity) and target (next time scalar) data ##
input_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1) #conentration
u_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1) #velocity
target_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1) #next time concentration

## Below are used when we are feeding multiple training datasets ##

#input2_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#u2_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#target2_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

#input3_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#u3_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#target3_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

#input4_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#u4_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#target4_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

#input5_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#u5_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#target5_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

#input6_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#u6_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
#target6_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

#input7_NN_integrate = zeros(Float32, ydim, 1, 1, nstep-1)
#u7_NN_integrate = zeros(Float32, ydim, 1, 1, nstep-1)
#target7_NN_integrate = zeros(Float32, ydim, 1, 1, nstep-1)

## When we use GPU to train ##

#input_NN_integrate |> gpu
#u_NN_integrate |> gpu
#target_NN_integrate |> gpu

## I just wanted to see if it is good ##
println("Good")

Good


In [6]:
## Now we store the scalar, velocity at the generated array ##

input_NN_integrate[:,1,1,:] = history[1][:,1:nstep-1]
u_NN_integrate[:,1,1,:] = history[2][:,1:nstep-1]
target_NN_integrate[:,1,1,:] = history[1][:,2:nstep]

#input2_NN_integrate[:,1,1,:] = history2[1][:,1:nstep-1]
#u2_NN_integrate[:,1,1,:] = history2[2][:,1:nstep-1]
#target2_NN_integrate[:,1,1,:] = history2[1][:,2:nstep]

#input3_NN_integrate[:,1,1,:] = history3[1][:,1:nstep-1]
#u3_NN_integrate[:,1,1,:] = history3[2][:,1:nstep-1]
#target3_NN_integrate[:,1,1,:] = history3[1][:,2:nstep]

#input4_NN_integrate[:,1,1,:] = history4[1][:,1:nstep-1]
#u4_NN_integrate[:,1,1,:] = history4[2][:,1:nstep-1]
#target4_NN_integrate[:,1,1,:] = history4[1][:,2:nstep]

#input5_NN_integrate[:,1,1,:] = history5[1][:,1:nstep-1]
#u5_NN_integrate[:,1,1,:] = history5[2][:,1:nstep-1]
#target5_NN_integrate[:,1,1,:] = history5[1][:,2:nstep]

#input6_NN_integrate[:,1,1,:] = history6[1][:,1:nstep-1]
#u6_NN_integrate[:,1,1,:] = history6[2][:,1:nstep-1]
#target6_NN_integrate[:,1,1,:] = history6[1][:,2:nstep]

#input7_NN_integrate[:,1,1,:] = history7[1][:,1:nstep-1]
#u7_NN_integrate[:,1,1,:] = history7[2][:,1:nstep-1]
#target7_NN_integrate[:,1,1,:] = history7[1][:,2:nstep]

224×2879 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.00182714  0.00183716  0.00184696
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.00533854  0.0053668   0.00539445
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.00862063  0.00866309  0.00870459
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0118458   0.0118979   0.0119486
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0150919   0.0151482   0.0152029
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0180649   0.0181185   0.0181702
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0204709   0.0205141   0.0205555
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.022319    0.0223457   0.0223708
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.023829    0.0238354   0.0238406
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0250864   0.025071    0.0250547
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0258404   0.025803    0.0257652
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0260894   0.0260333   0.0259774
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0260722   0.0260019   0.0259325
 ⋮                    

Now we create the convolutional neural networks. Depending on the problem complexity we could use different size of network. We could change the kernel size at the begining, or the number of layers.

In [24]:
## Three layer model
model = Chain( Conv((3,), 2 => 32, pad = SamePad(), gelu),
    Conv((3,), 32 => 32, pad = SamePad(), gelu),
    Conv((3,), 32 => 6, pad = SamePad(), tanh))

loss(x, y) = Flux.Losses.mae(model(x), y)

ps = Flux.params(model)

Params([[-0.23724414 -0.1847739; -0.2186838 -0.13725726; -0.18596105 -0.051771183;;; 0.107935704 0.09306445; -0.23084818 -0.05097895; -0.02000927 0.12976532;;; 0.027414659 0.17254841; -0.20021169 0.16721494; 0.17437194 0.16511054;;; … ;;; 0.05513018 -0.035334747; 0.15314297 -0.061656483; 0.13123395 0.13320887;;; 0.16111721 -0.03930819; 0.03960307 -0.044986144; -0.21798879 0.1565445;;; 0.063831136 0.11841493; -0.16270801 0.17739886; -0.062633 -0.096048824], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.07363952 -0.047109388 … -0.12786153 0.003514183; -0.17596187 -0.08074215 … 0.064750224 -0.13086778; -0.09424503 0.15709022 … -0.15967849 0.1111644;;; -0.17100489 0.16413192 … 0.12996615 -0.081389844; -0.033714738 -0.032853067 … -0.14265299 -0.055032805; 0.02673326 -0.015684191 … -0.12769084 0.011887202;;; -0.03100634 -0.12751737 … 0.16694982 0.09528412; 0.14384148 -0.15594721 … -0.17209105 0.16045; 0.010559387 -0.0273743

In [14]:
## Three layer model - 13-stencil
model = Chain( Conv((13,), 2 => 32, pad = SamePad(), gelu),
    Conv((13,), 32 => 32, pad = SamePad(), gelu),
    Conv((13,), 32 => 26, pad = SamePad(), tanh))

loss(x, y) = Flux.Losses.mae(model(x), y)

ps = Flux.params(model)

Params([[-0.009612143 -0.009659199; 0.044706717 0.089672185; … ; 0.056957096 -0.05944776; -0.012639276 -0.015488572;;; 0.06191108 0.11270316; -0.07864362 -0.09245483; … ; 0.10297047 -0.03236092; -0.026258748 -0.048672862;;; -0.050101694 -0.05139888; -0.08993016 0.016895887; … ; 0.050840553 0.11115066; 0.093233466 0.0017739873;;; … ;;; -0.09429093 -0.036797322; -0.032266762 0.080782376; … ; 0.09572192 0.062228806; 0.100657836 -0.11418092;;; 0.02613919 -0.0856348; -0.04975573 0.004815787; … ; -0.093202025 -0.074404754; -0.0044896705 0.035685305;;; -0.08045359 -0.009624699; 0.105840065 -0.008336247; … ; 0.055261154 -0.09312024; 0.033189096 -0.008104035], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.044070456 -0.008149819 … 0.07529017 0.037085187; 0.056104492 0.030698419 … -0.07855728 -0.049991682; … ; 0.06667938 -0.057261925 … 0.04981746 0.066125266; 0.047441065 -0.06792799 … 0.07857337 0.0020298928;;; 0.079462625 0.067

In [ ]:
## In case we need to use the larger size model - 4-layer here

model = Chain( Conv((31,), 2 => 64, pad = SamePad(), gelu),
    Conv((3,), 64 => 64, pad = SamePad(), gelu),
    Conv((3,), 64 => 64, pad = SamePad(), gelu),
    Conv((3,), 64 => 62, pad = SamePad(), tanh))

loss(x, y) = Flux.Losses.mae(model(x), y)

ps = Flux.params(model)

In [ ]:
## In case we need to use the larger size model - 4-layer with larger filters here

model = Chain( Conv((31,), 2 => 64, pad = SamePad(), gelu),
    Conv((3,), 64 => 128, pad = SamePad(), gelu),
    Conv((3,), 128 => 128, pad = SamePad(), gelu),
    Conv((3,), 128 => 62, pad = SamePad(), tanh))

loss(x, y) = Flux.Losses.mae(model(x), y)

ps = Flux.params(model)

In [ ]:
## In case we need to use the larger size model -5-layer here

model = Chain( Conv((31,), 2 => 64, pad = SamePad(), gelu),
    Conv((5,), 64 => 128, pad = SamePad(), gelu),
    Conv((5,), 128 => 128, pad = SamePad(), gelu),
    Conv((5,), 128 => 128, pad = SamePad(), gelu),
    Conv((5,), 128 => 62, pad = SamePad(), tanh))

loss(x, y) = Flux.Losses.mae(model(x), y)

ps = Flux.params(model)

In [7]:
## We could also load the trained model parameters.
#@load "spacing_phys_norm_outputs/CNN_DLR_60EPOCHS_16X1X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
@load "Train_outputs/01x01t/CNN_DLR_95EPOCHS_1X1X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
#model |> gpu
#ps = Flux.params(model) |> gpu;

In [ ]:
## Push the data in the format neural net could understand. 
Data = []
#Data2 = []
#Data3 = []
#Data4 = []
#Data5 = []
#Data6 = []
#Data7 = []

for i in 1:nstep-1
    push!(Data, (hcat(input_NN_integrate[:,:,:,i], u_NN_integrate[:,:,:,i]), target_NN_integrate[:,:,:,i]))
    #push!(Data2, (hcat(input2_NN_integrate[:,:,:,i], u2_NN_integrate[:,:,:,i]), target2_NN_integrate[:,:,:,i]))
    #push!(Data3, (hcat(input3_NN_integrate[:,:,:,i], u3_NN_integrate[:,:,:,i]), target3_NN_integrate[:,:,:,i]))
    #push!(Data4, (hcat(input4_NN_integrate[:,:,:,i], u4_NN_integrate[:,:,:,i]), target4_NN_integrate[:,:,:,i]))
    #push!(Data5, (hcat(input5_NN_integrate[:,:,:,i], u5_NN_integrate[:,:,:,i]), target5_NN_integrate[:,:,:,i]))
    #push!(Data6, (hcat(input6_NN_integrate[:,:,:,i], u6_NN_integrate[:,:,:,i]), target6_NN_integrate[:,:,:,i]))
    #push!(Data7, (hcat(input7_NN_integrate[:,:,:,i], u7_NN_integrate[:,:,:,i]), target7_NN_integrate[:,:,:,i]))
end

#Data |> gpu;

In [ ]:
## Training function for 10 time steps training

function my_custom_train!(ps, data, opt, dx, dt)
    
    local len = length(data)
    
    xdim = size(data[1][1],1);
    
    for i in 1:len
        gs = gradient(ps) do
            if  0 < i < len-8
                A = hcat([one_step_integrate(data[i][1][:,1,:], data[i][1][:,2,:], model, xdim, dx, dt)])
                for j in 1:9
                    x = A[j]
                    y = data[i+j][1][:,2]
                    A = hcat(A, [one_step_integrate(hcat(x, y)[:,1,:], hcat(x, y)[:,2,:], model, xdim, dx, dt)])
                end
                B = hcat([data[i+k-1][2] for k in 1:10])
                training_loss = mean(Flux.Losses.mae.(A, B))
            elseif len-8 ≤ i < len
                n = len - i
                A = hcat([one_step_integrate(data[i][1][:,1,:], data[i][1][:,2,:], model, xdim, dx, dt)])
                for j in 1:n
                    x = A[j]
                    y = data[i+j][1][:,2]
                    A = hcat(A, [one_step_integrate(hcat(x, y)[:,1,:], hcat(x, y)[:,2,:], model, xdim, dx, dt)])
                end
                B = hcat([data[i+k][2] for k in 0:n])
                training_loss = mean(Flux.Losses.mae.(A, B))
            elseif i == len
                A = hcat([one_step_integrate(data[i][1][:,1,:], data[i][1][:,2,:], model, xdim, dx, dt)])
                B = data[i][2]
                training_loss = mean(Flux.Losses.mae.(A, B))
            end
            return training_loss
        end
        Flux.update!(opt, ps, gs)
    end
end

In [ ]:
## Training loop. To see the progress, it printed out the number of epoch every time it's done.
answer = scalar1_GEOS_Array
model |> gpu
@time CUDA.@sync for i in 1:100
#@time for i in 1:100
    α = (1/(1+1*i))*0.01
    #α = 1e-3
    dx = Float32(27034.3*1)
    dt = 300*4
    my_custom_train!(ps, Data, ADAM(α, (0.9, 0.999)), dx, dt)
    #if i % 1 == 0
    #model |> cpu
    trial = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, dx, dt);
    dim = size(trial, 1);
    #writedlm( "Train_outputs/01x04t_13stencil/CNN_DLR_"*string(i)*"_01x04x_VL_GEOS_JAN_NASA_GMAO.csv", A, ',')
    #@save "Train_outputs/01x04t_13stencil/CNN_DLR_"*string(i)*"EPOCHS_01X04X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model    
    mae = StatsBase.L1dist(trial, reshape(answer,dim))/(dim)
    rmse = StatsBase.L2dist(trial, reshape(answer,dim))/sqrt(dim)
    r2 = Statistics.cor(trial, reshape(answer, dim))^2
    
    println(i, "   ", mae, "   ", rmse, "   ", r2)
    #model |> gpu
    #end
end
model |> cpu

In [ ]:
# Training loop for mulitple datasets
#@time CUDA.@sync for i in 1:200
@time for i in 1:500
    α = (1/(1+1*i))*0.01
    #α = 1e-3
    dx = Float32(27034.3*16)
    dx_29 = Float32(30276.7*16)
    dx_45 = Float32(24597.8*16)
    dy = Float32(27829.3*16)
    dt = 300*64
    my_custom_train!(ps, Data, ADAM(α, (0.9, 0.999)), dx, dt)
    my_custom_train!(ps, Data2, ADAM(α, (0.9, 0.999)), dx, dt)
    my_custom_train!(ps, Data3, ADAM(α, (0.9, 0.999)), dx, dt)
    my_custom_train!(ps, Data4, ADAM(α, (0.9, 0.999)), dx, dt)
    my_custom_train!(ps, Data5, ADAM(α, (0.9, 0.999)), dx_29, dt)
    my_custom_train!(ps, Data6, ADAM(α, (0.9, 0.999)), dx_45, dt)
    my_custom_train!(ps, Data7, ADAM(α, (0.9, 0.999)), dy, dt)
    #model |> cpu
    A1 = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, dx, dt);
    A2 = 1e-7*pgm_ml(input2_NN_integrate, u2_NN_integrate, model, xdim, nstep, dx, dt);
    A3 = 1e-7*pgm_ml(input3_NN_integrate, u3_NN_integrate, model, xdim, nstep, dx, dt);
    A4 = 1e-7*pgm_ml(input4_NN_integrate, u4_NN_integrate, model, xdim, nstep, dx, dt);
    A5 = 1e-7*pgm_ml(input5_NN_integrate, u5_NN_integrate, model, xdim, nstep, dx_29, dt);
    A6 = 1e-7*pgm_ml(input6_NN_integrate, u6_NN_integrate, model, xdim, nstep, dx_45, dt);
    A7 = 1e-7*pgm_ml(input7_NN_integrate, u7_NN_integrate, model, ydim, nstep, dy, dt);
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_U.csv", A1, ',')
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_U2.csv", A2, ',')
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_U3.csv", A3, ',')
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_U4.csv", A4, ',')
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_U5.csv", A5, ',')
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_U6.csv", A6, ',')
    writedlm( "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"_16x64x_VL_GEOS_JAN_NASA_GMAO_V.csv", A7, ',')
    @save "Train_outputs/16x64t_for2D_7data/CNN_DLR_"*string(i)*"EPOCHS_16X64X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
    println(i)
    #model |> gpu
end

In [ ]:
## The way we are using the trained model to return and save the output

@time A_8x16x = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, Float32(27034.3*8), 300*16);

In [ ]:
## The way we are using the trained model to return and save the output

@time A = 1e-7*pgm_ml(input7_NN_integrate, u7_NN_integrate, model, ydim, nstep, Float32(27829.3*16), 300*64);

In [20]:
## This cell is used to test the trained model again.
vel_GEOS_Array = readdlm( "Training_dataset/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_2x_64x.csv", ',', Float32) |> gpu;
scalar_GEOS_Array = readdlm("Training_dataset/VL_GEOS_Jan_2019_NASA_GMAO_10_U_2x_64x.csv", ',', Float32) |> gpu;
xdim = size(vel_GEOS_Array, 1) |> gpu;
nstep = size(vel_GEOS_Array, 2) |> gpu;

Random.seed!(1)
history = (scalar_GEOS_Array*Float32(1e7), vel_GEOS_Array/15) |> gpu;

input_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1) |> gpu
u_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1) |> gpu
target_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1) |> gpu

input_NN_integrate[:,1,1,:] = history[1][:,1:nstep-1] |> gpu
u_NN_integrate[:,1,1,:] = history[2][:,1:nstep-1] |> gpu
target_NN_integrate[:,1,1,:] = history[1][:,2:nstep] |> gpu

@load "Train_outputs/02x64t/CNN_DLR_221EPOCHS_02X64X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
model = gpu(model)
#ps = Flux.params(model) |> gpu;

Chain(
  Conv((17,), 2 => 32, gelu, pad=8),    # 1_120 parameters
  Conv((3,), 32 => 32, gelu, pad=1),    # 3_104 parameters
  Conv((3,), 32 => 34, tanh, pad=1),    # 3_298 parameters
)                   # Total: 6 arrays, 7_522 parameters, 1.211 KiB.

In [9]:
@time A = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, Float32(27034.3*1), 300*1);

  0.709195 seconds (512.49 k allocations: 935.674 MiB, 7.07% gc time)


In [10]:
## Evaluating the computational time

@benchmark A = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, Float32(27034.3*1), 300*1)

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  656.536 ms … 686.288 ms  ┊ GC (min … max): 3.66% … 4.19%
 Time  (median):     662.812 ms               ┊ GC (median):    3.65%
 Time  (mean ± σ):   666.384 ms ±  10.335 ms  ┊ GC (mean ± σ):  3.71% ± 0.20%

  █  ██   █        █           █       █                      █  
  █▁▁██▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  657 ms           Histogram: frequency by time          686 ms <

 Memory estimate: 935.67 MiB, allocs estimate: 512482.

In [ ]:
## Plot the r2 throughout the timesteps

begin
    r2 = zeros(Float32, nstep)
    for i in 1:nstep
        r2[i] = Statistics.cor(A[:,1,i], scalar1_GEOS_Array[:,i])^2
    end
    plot(Int(2880/nstep)*10/2880:Int(2880/nstep)*10/2880:10, r2, width=2, label=false, xlabel="time (days)", ylabel="r²", xlabelfontsize=14, ylabelfontsize=14,
        xtickfontsize=12, ytickfontsize=12, xlims=(0,10), ylims=(0.0,1.0))
end
savefig("r2_8x16t.png")

In [ ]:
## Calculating correlations in different time steps

Statistics.cor(A[:,1,1], 1e-7*input_NN_integrate[:,1,1,1])^2, Statistics.cor(A[:,1,960], 1e-7*input_NN_integrate[:,1,1,960])^2, Statistics.cor(A[:,1,1920], 1e-7*input_NN_integrate[:,1,1,1920])^2, Statistics.cor(A[:,1,2879], 1e-7*input_NN_integrate[:,1,1,2879])^2

In [ ]:
## Example of 1-D visualization with 8dx, 16dt case (t = 0 day). 
## You need to change the dimension if the resolution changes.

begin
    plot(-130.0+0.3125*4:0.3125*8:-60-0.3125*4, A[:,1,1]*1e9, label = "Learned", title = string("time: ", string(0), " days"), 
        xlabel="Longitude (°)", ylabel="Mixing ratio (ppb)", xlabelfontsize=18, ylabelfontsize=18, 
        xtickfontsize=14, ytickfontsize=14, titlefontsize=20, legendfontsize=14, width=4, ylims=(0.0, 1.2e2), legend = true,
         margin=2.5Plots.mm)
    plot!(-130.0+0.3125*4:0.3125*8:-60-0.3125*4, 1e2*input_NN_integrate[:,1,1,1], label = "Reference", width=4)
end

#savefig("8x16t_0day.png")

In [ ]:
## Example of 1-D visualization with 8dx, 16dt case (t = 10 days). 
## You need to change the dimension if the resolution changes.

begin
    plot(-130.0+0.3125*4:0.3125*8:-60-0.3125*4, 1e9*A[:,1,179], label = "Learned", title = string("time: ", string(10), " days"), 
        xtickfontsize=14, ytickfontsize=14, titlefontsize=20, width=4, ylims=(0.0, 1.2e2), legend = false, margin=2.5Plots.mm)
    plot!(-130.0+0.3125*4:0.3125*8:-60-0.3125*4, 1e2*input_NN_integrate[:,1,1,179], label = "Reference", width=4)
end

#savefig("8x16t_10day.png")

In [ ]:
## Example of 1-D visualization with 1dx, 1dt case (4 panels within 10 days - 0, 3.3, 6.6, and 10 days). 
## You need to change the dimension if the resolution changes.

begin
    xtf=12
    ytf=12
    p1=plot(-130.0+0.3125/2:0.3125*1:-60, 1e9*A[:,1,1], label = "Learned", xticks=[-120, -100, -80, -60], yticks=3,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.4e2), legend = false)
    plot!(-130.0+0.3125/2:0.3125*1:-60, 1e2*input_NN_integrate[:,1,1,1], label = "Reference", width=4)
    p2=plot(-130.0+0.3125/2:0.3125*1:-60, 1e9*A[:,1,960], label = "Learned", xticks=[-120, -100, -80, -60], yticks=3,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.4e2), legend = false)
    plot!(-130.0+0.3125/2:0.3125*1:-60, 1e2*input_NN_integrate[:,1,1,960], label = "Reference", width=4)
    p3=plot(-130.0+0.3125/2:0.3125*1:-60, 1e9*A[:,1,1920], label = "Learned", xticks=[-120, -100, -80, -60], yticks=3,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.4e2), legend = false)
    plot!(-130.0+0.3125/2:0.3125*1:-60, 1e2*input_NN_integrate[:,1,1,1920], label = "Reference", width=4)
    p4=plot(-130.0+0.3125/2:0.3125*1:-60, 1e9*A[:,1,2879], label = "Learned", xticks=[-120, -100, -80, -60], yticks=3,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.4e2), legend = false)
    plot!(-130.0+0.3125/2:0.3125*1:-60, 1e2*input_NN_integrate[:,1,1,2879], label = "Reference", width=4)
    
    plot(p1,p2,p3,p4, layout = (1,4), size=(1200, 200), bottommargin=5Plots.mm, rightmargin=2.5Plots.mm)
end

#savefig("1x1t_time_series.png")

In [ ]:
## Example of 1-D visualization with 8dx, 16dt case (4 panels within 10 days - 0, 3.3, 6.6, and 10 days). 
## You need to change the dimension if the resolution changes.

begin
    xtf=12
    ytf=12
    p1=plot(-130.0+0.3125*4:0.3125*8:-60, 1e9*A_8x16x[:,1,1], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*4:0.3125*8:-60, 1e2*input_NN_integrate[:,1,1,1], label = "Reference", width=4)
    p2=plot(-130.0+0.3125*4:0.3125*8:-60, 1e9*A_8x16x[:,1,60], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*4:0.3125*8:-60, 1e2*input_NN_integrate[:,1,1,60], label = "Reference", width=4)
    p3=plot(-130.0+0.3125*4:0.3125*8:-60, 1e9*A_8x16x[:,1,120], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*4:0.3125*8:-60, 1e2*input_NN_integrate[:,1,1,120], label = "Reference", width=4)
    p4=plot(-130.0+0.3125*4:0.3125*8:-60, 1e9*A_8x16x[:,1,179], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*4:0.3125*8:-60, 1e2*input_NN_integrate[:,1,1,179], label = "Reference", width=4)
    
    plot(p1,p2,p3,p4, layout = (1,4), size=(1200, 200), bottommargin=5Plots.mm, rightmargin=2.5Plots.mm)
end

savefig("8x16t_time_series.pdf")

In [ ]:
## Example of 1-D visualization with 16dx, 64dt case (4 panels within 10 days - 0, 3.3, 6.6, and 10 days). 
## You need to change the dimension if the resolution changes.

begin
    xtf=12
    ytf=12
    p1=plot(-130.0+0.3125*8:0.3125*16:-60, 1e9*A_16x64x[:,1,1], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*8:0.3125*16:-60, 1e2*input_NN_integrate[:,1,1,1], label = "Reference", width=4)
    p2=plot(-130.0+0.3125*8:0.3125*16:-60, 1e9*A_16x64x[:,1,15], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*8:0.3125*16:-60, 1e2*input_NN_integrate[:,1,1,15], label = "Reference", width=4)
    p3=plot(-130.0+0.3125*8:0.3125*16:-60, 1e9*A_16x64x[:,1,30], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*8:0.3125*16:-60, 1e2*input_NN_integrate[:,1,1,30], label = "Reference", width=4)
    p4=plot(-130.0+0.3125*8:0.3125*16:-60, 1e9*A_16x64x[:,1,44], label = "Learned", xticks=[-120, -100, -80, -60], yticks=4,
        xtickfontsize=xtf, ytickfontsize=ytf, width=4, xlims=(-130, -60), ylims=(0.0, 1.1e2), legend = false)
    plot!(-130.0+0.3125*8:0.3125*16:-60, 1e2*input_NN_integrate[:,1,1,44], label = "Reference", width=4)
    
    plot(p1,p2,p3,p4, layout = (1,4), size=(1200, 200), bottommargin=5Plots.mm, rightmargin=2.5Plots.mm)
end

savefig("16x64t_time_series.pdf")